In [2]:
import qforte as qf
from qforte import *
import numpy as np

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\cop}[1]{\hat{a}^{\dagger}_{#1}}
\newcommand{\aop}[1]{\hat{a}_{#1}}
$$

# Getting molecualr Hamiltonains 

In this tutorial we review how to obtain the molecular Hamiltonian, as well as other pertinent information, from the electronic structure backend. Currently, QForte accomplishes this goal via an interface to Psi4.

We assume that our system is described by the general two-body Hamiltonian
\begin{equation}
\hat{\mathcal{H}}_\mathrm{sq} = \sum_{pq} h_{pq} \cop{p} \aop{q}
+ \frac{1}{4} \sum_{pqrs}
v_{pqrs} \cop{p} \cop{q} \aop{s} \aop{r},
\end{equation}
where $\cop{p}$ ($\aop{q}$) is a fermionic annihilation (creation) operator, while $h_{pq}$ and $v_{pqrs}$
are one-electron and anti-symmetrized two-electron integrals, respectively. 

Our objective here is then to obtain (form the electron integrals generated by psi4) the qubit representation of this Hamiltonian
\begin{equation}
\hat{\mathcal{H}}_\mathrm{qb} = \sum_\ell \theta_\ell \hat{P}_\ell.
\end{equation}
Each operator $\hat{P}_\ell$ in $\hat{H}$ is a tensor product of $n_{\ell}$ Pauli operators (a Pauli string) that act on distinct qubits,
\begin{equation}
\hat{P}_\ell =  \bigotimes_{k=1}^{n_{\ell}} \sigma_{l_k}^{(j_k)},
\end{equation}
where $l_k \in \{ X, Y, Z\}$ labels the Pauli operator type and $j_k$ indicates the qubit upon which said operator is applied.   

## Specifying a geometry and runnng Psi4

Ultimately any calculation performed in QForte will begin (as is done using a classical electronic structure package) by specifying a (molecular) geometry.

In QForte this is all accomplished using the `system_factory` and `molecule` classes. 
To begin, one simply imports the appropriate modules and specifies a geometry.
Note that QForte uses a python list `_reference` (generated by instantiating the molecule) to define any initial states used in a calculation. It usually reflects the Hartree-Fock state such that the list has length equal to the number of spin-orbitals with the occupied spin-orbitals indicated by 1's and the unoccupied spin-orbitals indicated by 0's. By default they are ordered energetically by with alternating alpha and beta.   


> Specify molecular geometry for H2, run Psi4 as a backend, and get the QForte `molecule` object.

In [3]:
# Define the reference and geometry lists.
geom = [('H', (0., 0., 0.0)), ('H', (0., 0., 1.50))]

# Get the molecule object that now contains both the fermionic and qubit Hamiltonians.
H2mol = system_factory(build_type='psi4', mol_geometry=geom, basis='sto-3g')

 ==> Psi4 geometry <==
-------------------------
0  1
H  0.0  0.0  0.0
H  0.0  0.0  1.5
symmetry c1
units angstrom


From the `molecule` object, we have access to several key properties such as the second-quantized and qubit representations of the Hamiltonian, as well as various classical energies. 
By default `system_factory` will use Psi4 to run a SCF and FCI calculation, but one can optionally run additional backend calculations by passing the constructor keyword arguments. 

> Print the second-quantized Hamiltonian, the qubit Hamiltonian, Hartree-Fock energy, and FCI energy.

In [4]:
print('The QForte second-quantized Hamiltonain:')
print(H2mol.sq_hamiltonian, '\n\n')


print('The QForte qubit Hamiltonain:')
print(H2mol.hamiltonian, '\n\n')


print(f'The Hartree-Fock energy from Psi4:     {H2mol.hf_energy:12.10f}')
print(f'The FCI energy from Psi4:              {H2mol.fci_energy:12.10f}')

The QForte second-quantized Hamiltonain:
 +0.352785 ( )
 -0.908181 ( 0^ 0 )
 -0.908181 ( 1^ 1 )
 -0.552703 ( 1^ 0^ 1 0 )
 -0.000000 ( 1^ 0^ 2 1 )
 +0.000000 ( 1^ 0^ 3 0 )
 -0.229536 ( 1^ 0^ 3 2 )
 -0.665337 ( 2^ 2 )
 -0.330148 ( 2^ 0^ 2 0 )
 -0.000000 ( 2^ 1^ 1 0 )
 -0.559684 ( 2^ 1^ 2 1 )
 +0.229536 ( 2^ 1^ 3 0 )
 -0.665337 ( 3^ 3 )
 +0.000000 ( 3^ 0^ 1 0 )
 +0.229536 ( 3^ 0^ 2 1 )
 -0.559684 ( 3^ 0^ 3 0 )
 -0.330148 ( 3^ 1^ 3 1 )
 -0.229536 ( 3^ 2^ 1 0 )
 -0.583421 ( 3^ 2^ 3 2 )
 


The QForte qubit Hamiltonain:
+0.145855[Z3 Z2]
-0.035645[Z3]
+0.139921[Z2 Z1]
+0.057384[X3 Y2 Y1 X0]
+0.139921[Z3 Z0]
+0.093456[Z1]
+0.082537[Z3 Z1]
-0.035645[Z2]
+0.093456[Z0]
-0.057384[Y3 Y2 X1 X0]
+0.138176[Z1 Z0]
+0.082537[Z2 Z0]
+0.057384[Y3 X2 X1 Y0]
-0.057384[X3 X2 Y1 Y0]
-0.491786[] 


The Hartree-Fock energy from Psi4:     -0.9108735544
The FCI energy from Psi4:              -0.9981493534


## Computing the Hartree-Fock energy with QForte

Using the information in all the tutorials so far, it is possible to implement compute the Hartree-Fock energy using QForte.
This is accomplished by preparing the Hartree-Fock state on the quantum device and measuring the expectation value of the Hamiltonian (with integrals transformed into the molecular orbital basis).
We will need to construct a unitary circuit that sets the vacuum to the Hartree-Fock state for the H2 molecule.

> Build a HF state circuit ($\hat{U}_\mathrm{HF}$) for H2 and show that it constructs the correct state. 

In [8]:
# Initialize the circuit.
Uhf = qf.Circuit()
Uhf.add_gate(qf.gate('X', 0, 0))
Uhf.add_gate(qf.gate('X', 1, 1))

# Initialize a QuantumComputer
print('\nThe vaccume state:')
QC = qf.Computer(4)
print(QC)

# Set the QuantumComputer to the Hartree-Fock state using Uhf
print('\nThe Hartree-Fock state:')
QC.apply_circuit(Uhf)
print(QC)


The vaccume state:
Computer(
  (1.000000 +0.000000 i) |0000>
  (0.000000 +0.000000 i) |1000>
  (0.000000 +0.000000 i) |0100>
  (0.000000 +0.000000 i) |1100>
  (0.000000 +0.000000 i) |0010>
  (0.000000 +0.000000 i) |1010>
  (0.000000 +0.000000 i) |0110>
  (0.000000 +0.000000 i) |1110>
  (0.000000 +0.000000 i) |0001>
  (0.000000 +0.000000 i) |1001>
  (0.000000 +0.000000 i) |0101>
  (0.000000 +0.000000 i) |1101>
  (0.000000 +0.000000 i) |0011>
  (0.000000 +0.000000 i) |1011>
  (0.000000 +0.000000 i) |0111>
  (0.000000 +0.000000 i) |1111>
 )

The Hartree-Fock state:
Computer(
  (0.000000 +0.000000 i) |0000>
  (0.000000 +0.000000 i) |1000>
  (0.000000 +0.000000 i) |0100>
  (1.000000 +0.000000 i) |1100>
  (0.000000 +0.000000 i) |0010>
  (0.000000 +0.000000 i) |1010>
  (0.000000 +0.000000 i) |0110>
  (0.000000 +0.000000 i) |1110>
  (0.000000 +0.000000 i) |0001>
  (0.000000 +0.000000 i) |1001>
  (0.000000 +0.000000 i) |0101>
  (0.000000 +0.000000 i) |1101>
  (0.000000 +0.000000 i) |0011>
  (0

In QForte one can measure a (symmetric) expectation value of an operator with the function `QC.expectation(operaotr)`. Note that this function returns a complex value.

> Calculate $E_\mathrm{HF} = \bra{\Phi_\mathrm{HF}} \hat{\mathcal{H}} \ket{\Phi_\mathrm{HF}}$ using `QC.expectation(operaotr)`.

In [9]:
Ehf = np.real(QC.expectation(H2mol.hamiltonian))

print(f'The Hartree-Fock energy from Psi4:     {H2mol.hf_energy:12.10f}')
print(f'The Hartree-Fock energy from QForte:   {Ehf:12.10f}')

The Hartree-Fock energy from Psi4:     -0.9108735544
The Hartree-Fock energy from QForte:   -0.9108735544


## Implementing FCI with QForte

A useful exercise is to use what has been presented so far to implement FCI.
The basic idea of FCI is then to diagonalized the Hamiltonian matrix $\mathbf{H}$ in a basis $\{ \Phi_I \}$ of $N_I$ determinants, comprised of the Hartree-Fock determinant all possible $N_I - 1$ excited determinants.
The matrix elements of the Hamiltonian are then given by

\begin{equation}
H_{IJ} = \bra{\Phi_I}\hat{H}\ket{\Phi_J},
\end{equation}  

and Schrodinger's equation takes the form of a matrix eigenvalue problem $\mathbf{H}\mathbf{C} = \vec{E} \mathbf{C}$. 
The elements ($C_I$) of the eigenvectors $\vec{C}_I$, for example of the ground state ($C_I \in \vec{C}_0 = \mathbf{C}^{(0)}$), give coefficients in a determinantal expansion of the CI wave-function.
If the set of expansion determinants contains \textit{all} possible $N_\mathrm{FCI}$ combinations of $n$ electrons in $m$ states, then the the CI is considered \textit{full} and the basis spans the entire Hilbert space.
It follows that the full CI (FCI) state is given by    
\begin{equation}
\label{eq:fci}
\ket{\Psi_\mathrm{FCI}} = \sum_I^{N_\mathrm{FCI}} C_I \ket{\Phi_I}.
\end{equation}
The variational principal implies that the FCI state is the best possible approximation to the true ground state in a finite basis, capturing the entirety of electronic correlations.

One can use QForte to evaluate the matrix elements $H_{IJ} = \bra{\Phi_I}\hat{H}\ket{\Phi_J}$.
To acomplish this for H2 we will need a few intermediate steps, namely the construction of functions which are able to determine diagonal and off-diagonal matrix elements. 
Before we can do this, however, we will also need functions to consturct circutis that prepare states of the form.

> Define a funciton returns a circuit from a qubit list.

In [18]:
def get_UI(Phi_I):
    UI = Circuit()
    for j in range(len(Phi_I)):
        if Phi_I[j] == 1:
            UI.add_gate(qforte.gate('X', j, j))

    return UI

> Now a function that takes in two bit lists Phi_I and Phi_J (representing the qubit configurations of $\Phi_I$ and $\Phi_J$ that are of equal length and particle-number) and returns the circuit $\hat{U}_\mathrm{split}$. Test this circuit and print the resulting state. 

In [25]:
def get_Usplit(Phi_I, Phi_J):
    
    # Define the return circuit
    Usplit = qf.Circuit()
    
    if (len(Phi_I) != len(Phi_J)) or (sum(Phi_I) != sum(Phi_J)) or (Phi_I==Phi_J):
        raise ValueError("Phi_I and Phi_J must have the same length, the same particle-number, and be different states.")
        
    nqb = len(Phi_I)
        
    # Make list of dissimilar bits and which state has dissimilar bit set to 1.
    diff_bits = []
    ones_bits = []
    for i in range(nqb):
        if(Phi_I[i] != Phi_J[i]):
            if(Phi_I[i]):
                diff_bits.append((i,'Phi_I'))
            else:
                diff_bits.append((i,'Phi_J'))
        
        elif(Phi_I[i]==Phi_J[i]==1):       
            ones_bits.append(i)    
    
    # Phi_Idd the Hadamard gate that will split the state around the first dissimilar qubit
    Usplit.add_gate(qf.gate('H', diff_bits[0][0]))
    
    for k in range(1, len(diff_bits)):
        
        if diff_bits[k][1]==diff_bits[k-1][1]:
            Usplit.add_gate(qf.gate('cX',diff_bits[k][0] ,diff_bits[k-1][0] ))
        
        else:
            Usplit.add_gate(qf.gate('X',diff_bits[k-1][0]))
            Usplit.add_gate(qf.gate('cX',diff_bits[k][0], diff_bits[k-1][0]))
            Usplit.add_gate(qf.gate('X',diff_bits[k-1][0]))
    
    # Finally flip all the bits that are supposed to be 1.
    for p in ones_bits:
        Usplit.add_gate(qf.gate('X',p ,p ))
    
    return Usplit
    

Now we can define functions that can evaluate the diagonal and off-diagonal Hamiltonain elements. 
Evaluating the the diagonal matrix elements is very similar to measuring the Hartee-Fock energy above, but with different respect to other determinants

\begin{equation}
H_{II} = \bra{\Phi_I} \hat{H} \ket{\Phi_I}.
\end{equation}

Determination of the off-diagonal elements requires evaluation of three symmetric expectation values and is given by

\begin{equation}
H_{IJ} = \bra{\Omega_{IJ}} \hat{H} \ket{\Omega_{IJ}} - \frac{1}{2} H_{II} - \frac{1}{2} H_{JJ},
\end{equation}

where 

\begin{equation}
\ket{\Omega_{IJ}} = \frac{1}{\sqrt{2}} \ket{\Phi_I} + \frac{1}{\sqrt{2}} \ket{\Phi_J} = \hat{U}_\mathrm{split} \ket{\bar{0}}.
\end{equation}

is the split state.

> Define function to get the diagonal and off-diagonal elements of the Hamiltonain matrix


In [26]:
def get_HII(Phi_I, Ham):
    QC = Computer(len(Phi_I))
    UI = get_UI(Phi_I)
    QC.apply_circuit(UI)
    
    HII = np.real(QC.expectation(Ham))
    
    return HII

def get_HIJ(Phi_I, Phi_J, Ham):
    
    HII = get_HII(Phi_I, Ham)
    HJJ = get_HII(Phi_J, Ham)
    
    QC = Computer(len(Phi_I))
    Usplit = get_Usplit(Phi_I, Phi_J)
    QC.apply_circuit(Usplit)
    
    HIJ = np.real(QC.expectation(Ham)) - 0.5*HII - 0.5*HJJ
    
    return HIJ

> Make a list of FCI basis states for H2 and then use the functions you have defined to build and diagonalized the FCI Hamiltonain.

In [27]:
basis_states = [ [1,1,0,0], [0,1,1,0], [1,0,0,1], [0,0,1,1] ]
H_mat = np.zeros((len(basis_states), len(basis_states)))

# Populate the Hamiltonain matrix
for I, Phi_I in enumerate(basis_states):
    H_mat[I][I] = get_HII(Phi_I, H2mol.hamiltonian)
    for J, Phi_J in enumerate(basis_states):
        if I!=J:
            H_mat[I][J] = get_HIJ(Phi_I, Phi_J, H2mol.hamiltonian)
            
print(f'The Hartree-Fock energy from Psi4:     {H2mol.hf_energy:12.10f}')
print(f'The FCI energy from Psi4:              {H2mol.fci_energy:12.10f}')           
print(f"\n The Hamiltonain matrix: \n {H_mat}")

evals, evecs = np.linalg.eigh(H_mat)

print(f"\n The Eigenvalues: \n {evals}")
print(f"\n The Eigenvector: \n {evecs[:,0]}") # Test this

The Hartree-Fock energy from Psi4:     -0.9108735544
The FCI energy from Psi4:              -0.9981493534

 The Hamiltonain matrix: 
 [[-9.10873554e-01  0.00000000e+00  1.11022302e-16  2.29535936e-01]
 [ 0.00000000e+00 -6.61048845e-01 -2.29535936e-01  8.32667268e-17]
 [ 1.11022302e-16 -2.29535936e-01 -6.61048845e-01  8.32667268e-17]
 [ 2.29535936e-01  1.11022302e-16  1.11022302e-16 -3.94468303e-01]]

 The Eigenvalues: 
 [-0.99814935 -0.89058478 -0.43151291 -0.3071925 ]

 The Eigenvectors: 
 [-9.34713055e-01  0.00000000e+00  1.71902026e-16  3.55403298e-01]
